In [81]:
import  pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re 
import warnings
warnings.filterwarnings("ignore")


# 크롤링한 데이터를 데이터 프레임으로 만들기 위해 준비
columns = ["검색어", "순번", "한자","품사","설명","링크"]
df = pd.DataFrame(columns=columns)
excutable_path = "../chromedriver.exe"
driver = webdriver.Chrome(executable_path=excutable_path)
base_url = 'https://stdict.korean.go.kr'

def urlmake(links):
    source = links
    driver.get(source)
    req = driver.page_source
    soup = BeautifulSoup(req, "html.parser") 
    return soup

def listmake(word):
    num = isNone(word.select_one('.t_blue1 > sup'))
    china = isNone(word.select_one('dt> span'))
    poumsa =  isNone(word.select_one('dt'))
    content = isNone(word.select_one('font'))
    link = word.select_one('a').attrs['href']
    return num,china,poumsa,content,base_url +link

def isNone(word):
    if word != None and word.text != '':
        if len(word) > 3:
            index = startAndLast(re.sub('[\t\n\s]','',word.text))
            return re.sub('[\t\n\s]','',word.text)[index[0]:index[1]]
        return re.sub('[\t\n\s]','',word.text)
    else:
        return "Nope"

def startAndLast(word):
    ok = True
    start_index = 0
    last_index = 0
    for i in range(len(word)):
        if ok and word[i] == '「':
            start_index = i
        if word[i] == '」':
            last_index = i+1
            return start_index,last_index

        
        
        
        
search_data =  input("검색어를 입력해주세요.")


url = 'https://stdict.korean.go.kr/search/searchResult.do?pageSize=10&searchKeyword=' + str(search_data)

num = 0

search_soup = urlmake(url)

title = search_soup.select_one('.t_blue2')
if title != None:
    title = title.text.strip()
    words = search_soup.select('.result > li')
    for word in words:
        print(f'{columns[0]} : {title}')
        for i in range(len(listmake(word))):
            print(f'{columns[i+1]} : {listmake(word)[i]}') 
        print('----------------------------------------------------------------------------------------------')
        row = [title,*listmake(word)]
        series = pd.Series(row, index=df.columns)
        df = df.append(series, ignore_index=True)
else:
    text = search_soup.select_one('.no_result>li>div')
    if text == None:
        print(search_soup.select_one('.no_result').text)
    else:
        print(text.text)
        print('https://opendic.korean.go.kr/search/searchResult?query='+search_data)


"['ㅇㅇ', 'ㄹㄹ']"에 대한 검색 결과가 없습니다.



In [74]:
file = open('words.txt', 'rt', encoding ='utf8')
lines = file.readlines()
print(lines)

['학원\n', '학교\n', '역사\n', '수학\n', '소설']


In [75]:
f = open("words.txt", 'r', encoding = 'utf8')
lines = f.readlines()
for line in lines:
    line = line.strip()  
    print(line)
f.close()

학원
학교
역사
수학
소설


In [76]:
df

,검색어,순번,한자,품사,설명,링크
0,소설,1,(小舌),「명사」,하찮거나쓸모없는변설(辯舌).,https://stdict.korean.go.kr/search/searchView....
1,소설,2,(小雪),「명사」,"이십사절기의하나.입동과대설사이에들며,이때태양의황경(黃經)은240도가된다.11월22일...",https://stdict.korean.go.kr/search/searchView....
2,소설,3,(小說),「명사」,『문학』사실또는작가의상상력에바탕을두고허구적으로이야기를꾸며나간산문체의문학양식.일정한구...,https://stdict.korean.go.kr/search/searchView....
3,소설,4,(所說),「명사」,설명하는바.,https://stdict.korean.go.kr/search/searchView....
4,소설,5,(昭雪),「명사」,원통한죄나억울한누명따위를밝혀씻음.≒소석.,https://stdict.korean.go.kr/search/searchView....
5,소설,6,(素雪),「명사」,하얀눈.=백설.,https://stdict.korean.go.kr/search/searchView....
6,소설,7,(掃雪),「명사」,쌓인눈을치움.또는그런일.=제설.,https://stdict.korean.go.kr/search/searchView....
7,소설,8,(疏泄),「명사」,『한의』뭉쳐있는기운을풀어서소통시키는기능.,https://stdict.korean.go.kr/search/searchView....
8,소설,9,(騷說),「명사」,시끄럽게떠도는소문.,https://stdict.korean.go.kr/search/searchView....


In [78]:
df.to_csv('소설3.csv',index=False, encoding ='cp949') 